#### On my machine (Mike) I use conda env "trials"

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

#### Get protocol reference, vectorize and store in Qdrant

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Qdrant
from langchain_core.documents.base import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the document - here we are just using a protocol in a specific directory
file_path = './documents/protocol.pdf'
# file_path = './documents/consent.pdf'
separate_pages = []             
loader = PyMuPDFLoader(file_path)
page = loader.load()
separate_pages.extend(page)
print(f"Number of separate pages: {len(separate_pages)}")


Number of separate pages: 76


In [3]:

# OyMuPDFLoader loads pages into separate docs!
# This is a problem when we chunk because we only chunk individual
# documents.  We need ONE overall document so that the chunks can
# overlap between actual PDF pages.
document_string = ""
for page in separate_pages:
    document_string += page.page_content
print(f"Length of the document string: {len(document_string)}")


Length of the document string: 189621


In [4]:
import tiktoken

# CHOP IT UP
def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap = 200,
    length_function = tiktoken_len
)
text_chunks = text_splitter.split_text(document_string)
print(f"Number of chunks: {len(text_chunks)} ")
max_chunk_size = 0
for chunk in text_chunks:
    max_chunk_size = max(max_chunk_size, len(chunk))
print(f"Maximum chunk size: {max_chunk_size}")
document = [Document(page_content=chunk) for chunk in text_chunks]
print(f"Length of  document: {len(document)}")



Number of chunks: 28 
Maximum chunk size: 9891
Length of  document: 28


In [5]:
from qdrant_client import QdrantClient
import defaults

embedding_model = defaults.default_embedding_model
qdrant_url = defaults.default_url


In [6]:
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest
from langchain_qdrant import QdrantVectorStore
import hashlib

"""
This code creates a hash for every chunk and checks to see if that chunk already exists in the
vector database.  We only want one collection in Qdrant, but want to make sure that if a user
selects a document that has already been embedded and stored, it does not get stored again.  We
also add metadata for the document title, so that we can make our retriever focus on documents of
interest.  For example, after some usage, the application might have 20 documents for the user to 
select from.  We want the retriever to be exactly right for the documents that they selected.

This could also be useful if different versions of documents are in existence.  We would not want to
recreate a large vectorstore.  But the user could select the most recent version.
"""


def get_document_hash(doc_content):
    """Generate a unique hash for the document content."""
    return hashlib.md5(doc_content.encode()).hexdigest()

# Add a unique hash to your documents
for doc in document:
    doc.metadata['content_hash'] = get_document_hash(doc.page_content)

# Add the document title
for doc in document:
    doc.metadata['document_title'] = file_path.split('/')[-1]

client = QdrantClient(url=qdrant_url)

# If the collection exists, then we need to check to see if our document is already
# present, in which case we would not want to store it again.
if client.collection_exists("protocol_collection"):
    print("Collection exists")
    qdrant_vectorstore = QdrantVectorStore.from_existing_collection(
        embedding=embedding_model,
        collection_name="protocol_collection",
        url=qdrant_url
    )
    
    # Check for existing documents and only add new ones
    existing_hashes = set()
    new_docs = []
    
    # Get all existing hashes
    scroll_filter = rest.Filter(
        should=[
            rest.FieldCondition(
                key="metadata.content_hash",
                match=rest.MatchValue(value=doc.metadata['content_hash'])
            ) for doc in document
        ]
    )
    
    scroll_results = client.scroll(
        collection_name="protocol_collection",
        scroll_filter=scroll_filter,
        limit=len(document)  # Adjust this if you have a large number of documents
    )
    
    existing_hashes = set(point.payload.get('metadata', {}).get('content_hash') for point in scroll_results[0])
    
    for doc in document:
        if doc.metadata['content_hash'] not in existing_hashes:
            new_docs.append(doc)
    
    if new_docs:
        qdrant_vectorstore.add_documents(new_docs)
    
    print(f"Added {len(new_docs)} new documents")
    print(f"Skipped {len(existing_hashes)} existing documents")
else: 
    print("Collection does not exist")                           #So we go ahead and just add the documents
    qdrant_vectorstore = QdrantVectorStore.from_documents(
        documents=document,
        embedding=embedding_model,
        collection_name="protocol_collection",
        url=qdrant_url
    )


Collection exists
Added 0 new documents
Skipped 28 existing documents


In [7]:
from qdrant_client.http import models as rest

"""
This code sets up the search type but more importantly it has the filter
set up correctly.  We get a list of document titles that we want to include
in the filter, and pass it into the function, returning the retriever.

"""

def create_protocol_retriever(document_titles):
    return qdrant_vectorstore.as_retriever(
        # search_type='mmr',                                  #mmr is experiment for me
        search_kwargs={
            'filter': rest.Filter(
                must=[
                    rest.FieldCondition(
                        key="metadata.document_title",
                        match=rest.MatchAny(any=document_titles)
                    )
                ]
            ),
            'k': 15,                                         # for the mmr search it will only return k
            # 'fetch_k': 50,                                  # but will evaluate fetch_k candidates
        }
    )

# Usage example
# document_titles = ["consent.pdf", "protocol.pdf"]
document_titles = ["protocol.pdf"]
protocol_retriever = create_protocol_retriever(document_titles)



#### Test protocol retriever

In [8]:
risks = protocol_retriever.get_relevant_documents("Risks of study")
len(risks)
for i, doc in enumerate(risks, 1):
    print(f"Document {i}")
    print("-" * 50)
    print(doc.page_content[0:50])
    print("-" * 50)

/var/folders/zx/1kbq8dj123q1cq4b0tqqc64r0000gn/T/ipykernel_64287/111032867.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  risks = protocol_retriever.get_relevant_documents("Risks of study")


Document 1
--------------------------------------------------
blood cell count or anemia. Any blood sampling can
--------------------------------------------------
Document 2
--------------------------------------------------
PRECISE Protocol Version 1.07
Protocol Version Dat
--------------------------------------------------
Document 3
--------------------------------------------------
The Utah IRB has extensive experience with single 
--------------------------------------------------
Document 4
--------------------------------------------------
constitute the CPCCRN Biorepository Governance Com
--------------------------------------------------
Document 5
--------------------------------------------------
10.2.6
Monitoring Serious Adverse Events
A qualifi
--------------------------------------------------
Document 6
--------------------------------------------------
29
4.3.1
Eligibility to Receive Study Drug Doses .
--------------------------------------------------
Document 7
-----

In [9]:
# Check to see how big our prompt context is going to be
total_tokens=0
for risk in risks:
    total_tokens+= tiktoken_len(risk.page_content)
print(f"Tokens in context: {total_tokens}")


Tokens in context: 29294


In [10]:
# Create prompt
import prompts
from prompts import rag_prompt_template
from langchain.prompts import ChatPromptTemplate
rag_prompt = ChatPromptTemplate.from_template(prompts.rag_prompt_template)

In [11]:
# Create RAG chain
from defaults import default_llm
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

llm = default_llm

rag_chain = (
    {"context": itemgetter("question") | protocol_retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [12]:

summary_query = """ 
Please write a summary of the protocol that can be used as the introduction to an informed
consent document for a patient to participate in the study described in the protocol.  This
summary should be between 500 and 1000 words in length, and should be understandable by a normally
intelligent adult patient.  You must explain any medical or technical terms in a way
that a high school graduate can understand. The summary should briefly explain the study, the rationale for the study,
the risks and benefits of participation, and that participation is entirely voluntary.

Start the summary with a level 2 Markdown header (##) titled "Study Summary". Then continue with
the content without any further subheadings. Produce the entire summary in Markdown format so it 
can be nicely printed for the reader.

You should assume that the consent form is addressed to the parent of a potentially eligible
child, and the very beginnning of the summary should indicate that they are being invited to allow
their child to participate because the child is potentially eligible. State why their child is potentially
eligible.  

All the details of this introductory summary should be specific for this protocol.

"""

summary = rag_chain.invoke({"question":summary_query})

In [13]:
from IPython.display import display, Markdown
# Display the summary as Markdown
display(Markdown(summary))

## Study Summary

You are being invited to allow your child to participate in a research study because your child is potentially eligible due to their admission to a Pediatric or Cardiac Intensive Care Unit with sepsis-induced multiple organ dysfunction syndrome (MODS). This study, called the "PeRsonalizEd immunomodulation in pediatriC sepsIS-inducEd MODS (PRECISE)," aims to explore new ways to treat children with this condition by tailoring treatments to their specific immune responses.

Sepsis is a serious condition that occurs when the body's response to an infection causes injury to its own tissues and organs. In some cases, this can lead to MODS, where multiple organ systems in the body stop working properly. Current treatments for sepsis focus on antibiotics and supportive care, but there is a need for more personalized approaches that consider the unique immune responses of each child.

The PRECISE study is designed to test whether personalized treatments can improve outcomes for children with sepsis-induced MODS. The study involves two main trials: the GRACE-2 trial and the TRIPS trial. These trials will test different treatments based on your child's specific immune response. The GRACE-2 trial will focus on children with a suppressed immune response, while the TRIPS trial will focus on those with an overactive immune response.

Participation in this study is entirely voluntary. If you choose to allow your child to participate, they will undergo a series of tests to determine their immune response. Based on these results, they may receive one of the study treatments or be part of an observational group. The study treatments involve medications that are already approved for other uses but are being tested here for their potential to help children with sepsis-induced MODS.

There are potential risks and benefits to participating in this study. The risks include possible side effects from the study medications, such as fever, chills, or changes in blood cell counts. There is also a risk of discomfort from blood draws and other procedures. However, the study team will closely monitor your child to manage any side effects that occur.

The potential benefits of participating include the possibility of improved health outcomes for your child if the study treatments are effective. Additionally, the information gained from this study may help improve treatments for other children with sepsis-induced MODS in the future.

Your decision to allow your child to participate in this study is completely voluntary. You may choose not to participate, or you may withdraw your child from the study at any time without any penalty or loss of benefits to which your child is otherwise entitled. The care your child receives will not be affected by your decision.

If you decide to participate, you will be asked to sign a consent form, and the study team will provide you with more detailed information about the study procedures, potential risks, and benefits. You will have the opportunity to ask any questions you may have before making your decision.

Thank you for considering this important research study. Your participation could contribute to advancing medical knowledge and improving care for children with sepsis-induced MODS.

In [14]:
background_query = """ 
Please write a summary of the protocol that can be used as the background section of an informed
consent document for a patient to participate in the study described in the protocol.  This
summary should be between 500 and 1000 words in length, and should be understandable by a normally
intelligent adult patient.  You must explain any medical or technical terms in a way
that a high school graduate can understand. The summary should briefly explain why this patient is being
approached to be in the study, including a brief description of the disease that is being studied in the 
protocol, a description of the study interventions, and the scientific reasons that the investigators believe
the intervention might help the patient.  

Do not include the specific study procedures in this summary, because this will be presented in a different section of 
the informed consent document.  You also do not need to mention that participation is voluntary, nor
the specific risks and benefits of the study, because this information is being presented in a different
part of the informed consent document.

Start the summary with a level 2 Markdown header (##) titled "Background". Then continue with
the content without any further subheadings. Produce the entire summary in Markdown format so it 
can be nicely printed for the reader. 

All the details of this background summary should be specific for this protocol.

"""

background = rag_chain.invoke({"question":background_query})

display(Markdown(background))

## Background

You are being invited to participate in a research study focused on improving the treatment of a serious condition known as sepsis-induced multiple organ dysfunction syndrome (MODS) in children. Sepsis is a life-threatening condition that occurs when the body's response to an infection causes injury to its own tissues and organs. When sepsis leads to the failure of multiple organs, it is referred to as MODS. This condition is a major cause of illness and death in critically ill children, and current treatments are primarily supportive, such as providing antibiotics and managing symptoms.

The study you are being considered for is called the PRECISE study, which stands for PeRsonalizEd immunomodulation in pediatriC sepsIS-inducEd MODS. This study is part of a larger effort by the Collaborative Pediatric Critical Care Research Network to explore new ways to treat sepsis-induced MODS by tailoring treatments to the specific needs of each child based on their immune system's response to the infection.

The PRECISE study involves two main trials: the GRACE-2 trial and the TRIPS trial. These trials are designed to test different approaches to modulating, or adjusting, the immune system's response in children with sepsis-induced MODS. The goal is to improve outcomes by either boosting the immune system when it is too weak or calming it down when it is too active.

In the GRACE-2 trial, researchers are investigating the use of a drug called GM-CSF (granulocyte-macrophage colony-stimulating factor). GM-CSF is a naturally occurring protein in the body that stimulates the production of white blood cells, which are crucial for fighting infections. The researchers believe that by administering GM-CSF, they can help children whose immune systems are not responding adequately to the infection, a condition known as immunoparalysis. This approach aims to enhance the body's ability to fight off the infection and recover from organ dysfunction.

The TRIPS trial, on the other hand, is focused on children who have a different immune response characterized by excessive inflammation. In this trial, the drug anakinra is being tested. Anakinra is an anti-inflammatory medication that blocks the activity of a specific protein called interleukin-1, which plays a role in the inflammatory response. By using anakinra, the researchers hope to reduce harmful inflammation in children with sepsis-induced MODS, potentially leading to better recovery and outcomes.

The decision to include you in this study is based on specific criteria related to your immune system's response to sepsis. Researchers will conduct tests to determine your immune phenotype, which is a way of categorizing your immune response. Depending on the results, you may be eligible for one of the trials or be part of an observational group that helps researchers understand the condition better without receiving the study drugs.

The scientific rationale behind these interventions is grounded in previous research that has identified distinct immune response patterns in children with sepsis-induced MODS. By targeting these specific patterns with tailored treatments, the study aims to provide more effective care and improve the chances of recovery for children affected by this severe condition.

Your participation in this study could contribute to important scientific knowledge that may lead to better treatments for sepsis-induced MODS in the future.

In [15]:
number_of_participants_query = """ 
Please write a summary of the protocol that can be used 
for the "number of participants" section of the informed consent document.  This should include where the
study is being conducted (for example, at this hospital, or in a network, or in multiple hospitals), the funding source
(often the NIH), the total number of participants that are planned to be enrolled in the study,
and the total period of the time that the study is expected to enroll subjects. This summary should not require more than 200 words.

Start the summary with a level 2 Markdown header (##) titled "Number of Participants". Then continue with
the content without any further subheadings. Produce the entire summary in Markdown format so it 
can be nicely printed for the reader. 

All the details of this number of participants summary should be specific for this protocol.

"""

number_of_participants = rag_chain.invoke({"question":number_of_participants_query})

display(Markdown(number_of_participants))

## Number of Participants

This study is being conducted across multiple hospitals within the Collaborative Pediatric Critical Care Research Network (CPCCRN), which includes centers such as Nationwide Children’s Hospital, Children’s Hospital of Philadelphia, and Arkansas Children’s Hospital, among others. The study is funded by the Eunice Kennedy Shriver National Institute of Child Health and Human Development (NICHD). We plan to enroll up to 1,095 participants in total. Specifically, we anticipate enrolling 400 subjects into the GRACE-2 trial and 500 subjects into the TRIPS trial. The study is expected to enroll subjects over a period of four years. Participants will be critically ill children with sepsis-induced multiple organ dysfunction syndrome (MODS), and they will be immunophenotyped to determine eligibility for the interventional trials. This large-scale, multi-center study aims to improve outcomes through personalized, targeted immune modulation.

In [16]:
study_procedures_query = """ 
Please write a detailed summary of all the study procedures that will be carried out in this protocol.  This will
be used for the "study procedures" section of the informed consent document that the patient will read when deciding
whether to participate in the study, so it is important that all significant procedures be included.  
Make sure that everything will be understandable to the reader, who is not trained in medicine.  Do not write
the summary as if it is in third person - write it like you are speaking directly to the patient (i.e. use "you" instead 
of the "patient", with correct grammar of course.)  Do not include a welcome to the study, or discussion about
participation being voluntary, as that information is in a different part of the consent document.  Do not include
risks and benefits as these are presented in a different part of the consent document.  Please be detailed, as it is 
important that the patient understand each procedure.
The length of this summary is usually
2000 to 3000 words.

Start the summary with a level 2 Markdown header (##) titled "Study Procedures", and then continue the section with subheadings
that will help organize the information for the reader.  Do not go more than two subheadings deep.

All the details of study procedures should be specific to this protocol.

"""

study_procedures = rag_chain.invoke({"question":study_procedures_query})
display(Markdown(study_procedures))

## Study Procedures

### Screening and Enrollment

When you are admitted to the pediatric or cardiac intensive care unit, you will be evaluated to see if you are eligible for our study. This evaluation will involve checking if you meet certain criteria related to your health condition. If you are eligible, we will ask your legal guardian(s) for permission to include you in the study. This initial step is crucial as it helps us determine if you can participate in the study based on your current health status.

### Immune Phenotyping

If you are eligible and your guardian consents, we will perform a process called immune phenotyping. This involves taking a small blood sample from you on the second day after the onset of your condition, known as sepsis-induced multiple organ dysfunction syndrome (MODS). The blood sample will be used to measure certain immune responses and inflammatory markers, such as TNFα production capacity, serum ferritin levels, and C-reactive protein (CRP) levels. These tests help us understand your immune system's response to the illness and determine the best course of action for your treatment within the study.

### Randomization and Study Drug Administration

Based on the results of your immune phenotyping, you may be eligible to receive a study drug. If you qualify, you will be randomly assigned to receive either the study drug or a placebo. This process is called randomization and ensures that the study results are unbiased. The study drug will be administered intravenously, meaning it will be given through a vein, over a period of time. The specific drug and dosage will depend on your immune response and inflammation levels as determined by the phenotyping.

### Follow-up Immune Function Testing

Throughout the study, we will continue to monitor your immune function. This involves taking additional blood samples at specific intervals to measure the same markers as before. These follow-up tests help us track how your immune system is responding to the study drug and whether there are any changes in your condition. The samples will be processed and analyzed to provide insights into your treatment progress.

### Biorepository Sample Collection

As part of the study, we will collect additional biological samples, such as serum, plasma, and possibly endotracheal aspirates if you are intubated. These samples will be stored in a biorepository for future analysis related to the study. The collection of these samples is important for understanding the broader effects of the study drug and the underlying biology of your condition.

### Data Collection and Monitoring

Throughout your participation in the study, we will collect various types of data, including demographic information, eligibility data, and details about your health status and treatment. This data will be securely stored and used to evaluate the effectiveness and safety of the study drug. We will also monitor for any adverse events or changes in your condition that may occur during the study.

### Discontinuation or Withdrawal

You or your guardian may choose to discontinue participation in the study at any time. If you decide to stop participating, we will cease all study-related procedures and data collection. However, we will continue to review any data collected prior to your withdrawal to ensure the integrity of the study.

### Conclusion

These procedures are designed to ensure that we gather accurate and comprehensive information about your condition and the effects of the study drug. Your participation is valuable in helping us understand and improve treatments for sepsis-induced MODS in children.

In [17]:
alt_procedures_query = """ 
Please write a  summary of alternatives to participation in this study.  An example is:

" Your participation in this study is voluntary.  It is not necessary to be in this study to get care for
your illness.  Monitoring of immune function is not currently done as part of routine ICU care.  There are no
other treatments designed to increase immune function of reduce inflammation that are routinely used in 
children with sepsis."

Note that this example is purely an example, and your summary must be specific to the protocol. The summary should
be easily understandable by medically untrained readers.  This section is usually less than 500 words in length.


Start the summary with a level 2 Markdown header (##) titled "Alternative Procedures", and then continue with
the content without any further subheadings. Produce the entire summary in Markdown format so it 
can be nicely printed for the reader. 


"""

alt_procedures = rag_chain.invoke({"question":alt_procedures_query})
display(Markdown(alt_procedures))

## Alternative Procedures

Your participation in this study is entirely voluntary. You do not need to be part of this study to receive high-quality care for sepsis-induced multiple organ dysfunction syndrome (MODS). The medical team in the Pediatric Intensive Care Unit (PICU) or Cardiac Intensive Care Unit (CICU) will continue to provide state-of-the-art critical care, which includes monitoring and treating sepsis and its complications. This care is based on the best available medical practices and guidelines.

Currently, the monitoring of immune function and the specific treatments being studied, such as personalized immunomodulation, are not part of routine care in the ICU. These investigational treatments aim to improve outcomes by targeting the immune response, but they are not yet standard practice. If you choose not to participate, your child will still receive all the necessary medical interventions to manage sepsis and MODS, including antibiotics, fluid resuscitation, and supportive therapies as needed.

If at any point you decide to withdraw from the study, you can do so without any impact on the care your child receives. The decision to participate or not will not affect your relationship with the medical staff or the quality of care provided. Your child's health and well-being remain the top priority, and the medical team will continue to use all available resources to ensure the best possible outcomes.

In [18]:
risks_query = """ 
Please write a detailed summary of the risks of participating in  the study.  This will be used for the
"Risks" section of the informed consent document.  It is important that all significant risks of study
participation are described in detail. The summary must be easily readable by untrained readers, so provide
definitions of technical or medical terms.  Address all the risks by speaking to the patient, not by passively
mentioning risks to "the patient".  Especially include risks that are associated with the study interventions such
as drugs or devices, or associated with testing that is done as part of the study.  Also include
the risks associated with data collection, and also mention "unforeseable risks".

The length of this risk summary is usually
2000 to 3000 words.

Start the summary with a level 2 Markdown header (##) titled "Risks", and then continue the section with subheadings
that will help organize the information for the reader.  Do not go more than two subheadings deep.

All the details of study risks should be specific to this protocol.

"""

risks = rag_chain.invoke({"question":risks_query})
display(Markdown(risks))

## Risks

Participating in this study involves several risks that you should be aware of before deciding to take part. These risks are associated with the study drugs, procedures, and data collection. While we have taken steps to minimize these risks, it is important to understand them fully. Additionally, there may be unforeseeable risks that we cannot predict at this time.

### Risks Associated with Study Drugs

#### GM-CSF (Granulocyte Macrophage-Colony Stimulating Factor)

GM-CSF is a medication used to stimulate the production of white blood cells. It is generally safe, but there are some potential risks:

- **Fever and Chills**: You may experience fever and chills as your body responds to the medication.
- **Bone Pain**: Some participants report bone pain, which can be uncomfortable but is usually manageable with pain relief.
- **Dyspnea (Shortness of Breath)**: You might feel short of breath, which should be reported to the study team immediately.
- **Tachycardia (Increased Heart Rate)**: An increased heart rate can occur, which may require monitoring.
- **Peripheral Edema (Swelling)**: Swelling in the limbs can happen, especially if the drug is administered quickly. We will administer it slowly to minimize this risk.
- **Pericardial Effusion (Fluid Around the Heart)**: This is rare but can occur, leading to discomfort or more serious heart issues.
- **Hypersensitivity Reactions**: Allergic reactions, including rash or more severe reactions, can occur. Immediate medical attention will be provided if this happens.

#### Anakinra

Anakinra is used to reduce inflammation by blocking a specific protein in the body. Here are the potential risks:

- **Increased Risk of Infections**: Anakinra can increase the risk of infections, as it affects the immune system. We will monitor you closely for signs of infection.
- **Neutropenia (Low White Blood Cell Count)**: This condition can make you more susceptible to infections. Regular blood tests will be conducted to monitor your white blood cell levels.
- **Hypersensitivity Reactions**: Allergic reactions, including severe reactions like anaphylaxis, are possible. We will be prepared to treat any allergic reactions immediately.

### Risks Associated with Biological Sampling

#### Blood Sampling

Blood samples will be taken several times during the study. The risks associated with blood sampling include:

- **Anemia (Low Red Blood Cell Count)**: Frequent blood draws can lead to anemia, which may cause fatigue and weakness. We will limit the amount of blood drawn to minimize this risk.
- **Pain and Bruising**: You may experience pain and bruising at the site where blood is drawn. This is usually temporary.
- **Infection**: There is a small risk of infection at the site of the blood draw. We will use sterile techniques to minimize this risk.

#### Venipuncture and Capillary Sticks

Additional blood samples may be needed, which could require venipuncture (inserting a needle into a vein) or capillary sticks (pricking the skin). These procedures can cause:

- **Pain and Discomfort**: You may feel pain or discomfort during the procedure.
- **Bruising and Bleeding**: Bruising and minor bleeding can occur at the site of the needle or stick.
- **Infection**: As with blood draws, there is a small risk of infection.

### Risks Associated with Data Collection

#### Loss of Confidentiality

While we take extensive measures to protect your personal information, there is a minimal risk of loss of confidentiality. We use secure data management systems to protect your data, but no system is completely foolproof.

### Unforeseeable Risks

As with any clinical study, there may be risks that we cannot predict at this time. These unforeseeable risks could arise from the study drugs, procedures, or other aspects of the study. We will monitor your health closely and provide any necessary medical care if unexpected issues arise.

### Conclusion

Your safety is our top priority, and we will take all necessary precautions to minimize these risks. If you have any questions or concerns about the risks involved in this study, please do not hesitate to ask the study team. Your participation is voluntary, and you can withdraw from the study at any time if you feel uncomfortable with the risks.

In [19]:
benefits_query = """ 
Please write a  summary of the potential benefits of participating in  the study.  This will be used for the
"Benefits" section of the informed consent document.  The summary should include potential benefits for the patient
(addressed as "you"), and potential benefits for others.  Since this is a research study and it is
not known if the intervention is helpful, it is important to not overstate potential benefits for the patient.

The length of this risk summary is usually
500 to 750 words.

Start the summary with a level 2 Markdown header (##) titled "Benefits",  and then continue 
with a subheading for "Potential
Benefits for You" and another subheading for "Potential Benefits for Others".

All the information of study benefits should be specific to this protocol.

"""

benefits = rag_chain.invoke({"question":benefits_query})
display(Markdown(benefits))

## Benefits

### Potential Benefits for You

Participating in this study may offer you potential benefits, although it is important to understand that the effectiveness of the interventions being tested is not yet known. The primary goal of this study is to explore whether personalized immunomodulation can improve outcomes for children with sepsis-induced multiple organ dysfunction syndrome (MODS). If you are enrolled in the GRACE-2 trial, you may receive GM-CSF, which is being investigated for its potential to reverse immunoparalysis and enhance your immune response. Alternatively, if you are part of the TRIPS trial, you may receive anakinra, which is being studied for its ability to reduce inflammation and improve organ function. These interventions could potentially lead to a reduction in the severity and duration of organ dysfunction, and in some cases, may contribute to a quicker recovery. However, it is crucial to note that these benefits are not guaranteed, and the study is designed to determine the effectiveness of these treatments.

### Potential Benefits for Others

Your participation in this study could contribute significantly to the advancement of medical knowledge and the development of new treatments for sepsis-induced MODS in children. By helping researchers understand how personalized immunomodulation affects immune function and inflammation, you are playing a vital role in potentially improving future care for children facing similar health challenges. The data collected from this study may lead to more effective, targeted therapies that can reduce mortality and morbidity associated with sepsis-induced MODS. Additionally, the insights gained from this research could inform clinical practices and guidelines, ultimately benefiting a broader population of critically ill children. Your involvement is invaluable in paving the way for future innovations in pediatric critical care.

In [20]:
# Assuming you have already generated these summaries

# import makeWord
# from makeWord import makeWord
# summaries = [
#     summary,
#     background,
#     number_of_participants,
#     study_procedures,
#     alt_procedures,
#     risks,
#     benefits
# ]
#     # Combine all summaries into one string
# combined_markdown = "\n\n---\n\n".join(summaries)


In [21]:
# display(Markdown(combined_markdown))

In [22]:
# makeWord(combined_markdown, "My Newest Document")

In [23]:
from IPython.display import display, Markdown
import os

# Assuming you have already generated these summaries
summaries = [
    summary,
    background,
    number_of_participants,
    study_procedures,
    alt_procedures,
    risks,
    benefits
]

# Combine all summaries into one string
combined_markdown = "\n\n---\n\n".join(summaries)

# Add a title to the combined document
full_document = "# Full Informed Consent Document\n\n" + combined_markdown

# Display the full document in the notebook
display(Markdown(full_document))

# Save the Markdown content to a file
file_path = 'informed_consent_document.md'
with open(file_path, 'w', encoding='utf-8') as f:
    f.write(full_document)

print(f"Markdown document has been created: {os.path.abspath(file_path)}")

# Full Informed Consent Document

## Study Summary

You are being invited to allow your child to participate in a research study because your child is potentially eligible due to their admission to a Pediatric or Cardiac Intensive Care Unit with sepsis-induced multiple organ dysfunction syndrome (MODS). This study, called the "PeRsonalizEd immunomodulation in pediatriC sepsIS-inducEd MODS (PRECISE)," aims to explore new ways to treat children with this condition by tailoring treatments to their specific immune responses.

Sepsis is a serious condition that occurs when the body's response to an infection causes injury to its own tissues and organs. In some cases, this can lead to MODS, where multiple organ systems in the body stop working properly. Current treatments for sepsis focus on antibiotics and supportive care, but there is a need for more personalized approaches that consider the unique immune responses of each child.

The PRECISE study is designed to test whether personalized treatments can improve outcomes for children with sepsis-induced MODS. The study involves two main trials: the GRACE-2 trial and the TRIPS trial. These trials will test different treatments based on your child's specific immune response. The GRACE-2 trial will focus on children with a suppressed immune response, while the TRIPS trial will focus on those with an overactive immune response.

Participation in this study is entirely voluntary. If you choose to allow your child to participate, they will undergo a series of tests to determine their immune response. Based on these results, they may receive one of the study treatments or be part of an observational group. The study treatments involve medications that are already approved for other uses but are being tested here for their potential to help children with sepsis-induced MODS.

There are potential risks and benefits to participating in this study. The risks include possible side effects from the study medications, such as fever, chills, or changes in blood cell counts. There is also a risk of discomfort from blood draws and other procedures. However, the study team will closely monitor your child to manage any side effects that occur.

The potential benefits of participating include the possibility of improved health outcomes for your child if the study treatments are effective. Additionally, the information gained from this study may help improve treatments for other children with sepsis-induced MODS in the future.

Your decision to allow your child to participate in this study is completely voluntary. You may choose not to participate, or you may withdraw your child from the study at any time without any penalty or loss of benefits to which your child is otherwise entitled. The care your child receives will not be affected by your decision.

If you decide to participate, you will be asked to sign a consent form, and the study team will provide you with more detailed information about the study procedures, potential risks, and benefits. You will have the opportunity to ask any questions you may have before making your decision.

Thank you for considering this important research study. Your participation could contribute to advancing medical knowledge and improving care for children with sepsis-induced MODS.

---

## Background

You are being invited to participate in a research study focused on improving the treatment of a serious condition known as sepsis-induced multiple organ dysfunction syndrome (MODS) in children. Sepsis is a life-threatening condition that occurs when the body's response to an infection causes injury to its own tissues and organs. When sepsis leads to the failure of multiple organs, it is referred to as MODS. This condition is a major cause of illness and death in critically ill children, and current treatments are primarily supportive, such as providing antibiotics and managing symptoms.

The study you are being considered for is called the PRECISE study, which stands for PeRsonalizEd immunomodulation in pediatriC sepsIS-inducEd MODS. This study is part of a larger effort by the Collaborative Pediatric Critical Care Research Network to explore new ways to treat sepsis-induced MODS by tailoring treatments to the specific needs of each child based on their immune system's response to the infection.

The PRECISE study involves two main trials: the GRACE-2 trial and the TRIPS trial. These trials are designed to test different approaches to modulating, or adjusting, the immune system's response in children with sepsis-induced MODS. The goal is to improve outcomes by either boosting the immune system when it is too weak or calming it down when it is too active.

In the GRACE-2 trial, researchers are investigating the use of a drug called GM-CSF (granulocyte-macrophage colony-stimulating factor). GM-CSF is a naturally occurring protein in the body that stimulates the production of white blood cells, which are crucial for fighting infections. The researchers believe that by administering GM-CSF, they can help children whose immune systems are not responding adequately to the infection, a condition known as immunoparalysis. This approach aims to enhance the body's ability to fight off the infection and recover from organ dysfunction.

The TRIPS trial, on the other hand, is focused on children who have a different immune response characterized by excessive inflammation. In this trial, the drug anakinra is being tested. Anakinra is an anti-inflammatory medication that blocks the activity of a specific protein called interleukin-1, which plays a role in the inflammatory response. By using anakinra, the researchers hope to reduce harmful inflammation in children with sepsis-induced MODS, potentially leading to better recovery and outcomes.

The decision to include you in this study is based on specific criteria related to your immune system's response to sepsis. Researchers will conduct tests to determine your immune phenotype, which is a way of categorizing your immune response. Depending on the results, you may be eligible for one of the trials or be part of an observational group that helps researchers understand the condition better without receiving the study drugs.

The scientific rationale behind these interventions is grounded in previous research that has identified distinct immune response patterns in children with sepsis-induced MODS. By targeting these specific patterns with tailored treatments, the study aims to provide more effective care and improve the chances of recovery for children affected by this severe condition.

Your participation in this study could contribute to important scientific knowledge that may lead to better treatments for sepsis-induced MODS in the future.

---

## Number of Participants

This study is being conducted across multiple hospitals within the Collaborative Pediatric Critical Care Research Network (CPCCRN), which includes centers such as Nationwide Children’s Hospital, Children’s Hospital of Philadelphia, and Arkansas Children’s Hospital, among others. The study is funded by the Eunice Kennedy Shriver National Institute of Child Health and Human Development (NICHD). We plan to enroll up to 1,095 participants in total. Specifically, we anticipate enrolling 400 subjects into the GRACE-2 trial and 500 subjects into the TRIPS trial. The study is expected to enroll subjects over a period of four years. Participants will be critically ill children with sepsis-induced multiple organ dysfunction syndrome (MODS), and they will be immunophenotyped to determine eligibility for the interventional trials. This large-scale, multi-center study aims to improve outcomes through personalized, targeted immune modulation.

---

## Study Procedures

### Screening and Enrollment

When you are admitted to the pediatric or cardiac intensive care unit, you will be evaluated to see if you are eligible for our study. This evaluation will involve checking if you meet certain criteria related to your health condition. If you are eligible, we will ask your legal guardian(s) for permission to include you in the study. This initial step is crucial as it helps us determine if you can participate in the study based on your current health status.

### Immune Phenotyping

If you are eligible and your guardian consents, we will perform a process called immune phenotyping. This involves taking a small blood sample from you on the second day after the onset of your condition, known as sepsis-induced multiple organ dysfunction syndrome (MODS). The blood sample will be used to measure certain immune responses and inflammatory markers, such as TNFα production capacity, serum ferritin levels, and C-reactive protein (CRP) levels. These tests help us understand your immune system's response to the illness and determine the best course of action for your treatment within the study.

### Randomization and Study Drug Administration

Based on the results of your immune phenotyping, you may be eligible to receive a study drug. If you qualify, you will be randomly assigned to receive either the study drug or a placebo. This process is called randomization and ensures that the study results are unbiased. The study drug will be administered intravenously, meaning it will be given through a vein, over a period of time. The specific drug and dosage will depend on your immune response and inflammation levels as determined by the phenotyping.

### Follow-up Immune Function Testing

Throughout the study, we will continue to monitor your immune function. This involves taking additional blood samples at specific intervals to measure the same markers as before. These follow-up tests help us track how your immune system is responding to the study drug and whether there are any changes in your condition. The samples will be processed and analyzed to provide insights into your treatment progress.

### Biorepository Sample Collection

As part of the study, we will collect additional biological samples, such as serum, plasma, and possibly endotracheal aspirates if you are intubated. These samples will be stored in a biorepository for future analysis related to the study. The collection of these samples is important for understanding the broader effects of the study drug and the underlying biology of your condition.

### Data Collection and Monitoring

Throughout your participation in the study, we will collect various types of data, including demographic information, eligibility data, and details about your health status and treatment. This data will be securely stored and used to evaluate the effectiveness and safety of the study drug. We will also monitor for any adverse events or changes in your condition that may occur during the study.

### Discontinuation or Withdrawal

You or your guardian may choose to discontinue participation in the study at any time. If you decide to stop participating, we will cease all study-related procedures and data collection. However, we will continue to review any data collected prior to your withdrawal to ensure the integrity of the study.

### Conclusion

These procedures are designed to ensure that we gather accurate and comprehensive information about your condition and the effects of the study drug. Your participation is valuable in helping us understand and improve treatments for sepsis-induced MODS in children.

---

## Alternative Procedures

Your participation in this study is entirely voluntary. You do not need to be part of this study to receive high-quality care for sepsis-induced multiple organ dysfunction syndrome (MODS). The medical team in the Pediatric Intensive Care Unit (PICU) or Cardiac Intensive Care Unit (CICU) will continue to provide state-of-the-art critical care, which includes monitoring and treating sepsis and its complications. This care is based on the best available medical practices and guidelines.

Currently, the monitoring of immune function and the specific treatments being studied, such as personalized immunomodulation, are not part of routine care in the ICU. These investigational treatments aim to improve outcomes by targeting the immune response, but they are not yet standard practice. If you choose not to participate, your child will still receive all the necessary medical interventions to manage sepsis and MODS, including antibiotics, fluid resuscitation, and supportive therapies as needed.

If at any point you decide to withdraw from the study, you can do so without any impact on the care your child receives. The decision to participate or not will not affect your relationship with the medical staff or the quality of care provided. Your child's health and well-being remain the top priority, and the medical team will continue to use all available resources to ensure the best possible outcomes.

---

## Risks

Participating in this study involves several risks that you should be aware of before deciding to take part. These risks are associated with the study drugs, procedures, and data collection. While we have taken steps to minimize these risks, it is important to understand them fully. Additionally, there may be unforeseeable risks that we cannot predict at this time.

### Risks Associated with Study Drugs

#### GM-CSF (Granulocyte Macrophage-Colony Stimulating Factor)

GM-CSF is a medication used to stimulate the production of white blood cells. It is generally safe, but there are some potential risks:

- **Fever and Chills**: You may experience fever and chills as your body responds to the medication.
- **Bone Pain**: Some participants report bone pain, which can be uncomfortable but is usually manageable with pain relief.
- **Dyspnea (Shortness of Breath)**: You might feel short of breath, which should be reported to the study team immediately.
- **Tachycardia (Increased Heart Rate)**: An increased heart rate can occur, which may require monitoring.
- **Peripheral Edema (Swelling)**: Swelling in the limbs can happen, especially if the drug is administered quickly. We will administer it slowly to minimize this risk.
- **Pericardial Effusion (Fluid Around the Heart)**: This is rare but can occur, leading to discomfort or more serious heart issues.
- **Hypersensitivity Reactions**: Allergic reactions, including rash or more severe reactions, can occur. Immediate medical attention will be provided if this happens.

#### Anakinra

Anakinra is used to reduce inflammation by blocking a specific protein in the body. Here are the potential risks:

- **Increased Risk of Infections**: Anakinra can increase the risk of infections, as it affects the immune system. We will monitor you closely for signs of infection.
- **Neutropenia (Low White Blood Cell Count)**: This condition can make you more susceptible to infections. Regular blood tests will be conducted to monitor your white blood cell levels.
- **Hypersensitivity Reactions**: Allergic reactions, including severe reactions like anaphylaxis, are possible. We will be prepared to treat any allergic reactions immediately.

### Risks Associated with Biological Sampling

#### Blood Sampling

Blood samples will be taken several times during the study. The risks associated with blood sampling include:

- **Anemia (Low Red Blood Cell Count)**: Frequent blood draws can lead to anemia, which may cause fatigue and weakness. We will limit the amount of blood drawn to minimize this risk.
- **Pain and Bruising**: You may experience pain and bruising at the site where blood is drawn. This is usually temporary.
- **Infection**: There is a small risk of infection at the site of the blood draw. We will use sterile techniques to minimize this risk.

#### Venipuncture and Capillary Sticks

Additional blood samples may be needed, which could require venipuncture (inserting a needle into a vein) or capillary sticks (pricking the skin). These procedures can cause:

- **Pain and Discomfort**: You may feel pain or discomfort during the procedure.
- **Bruising and Bleeding**: Bruising and minor bleeding can occur at the site of the needle or stick.
- **Infection**: As with blood draws, there is a small risk of infection.

### Risks Associated with Data Collection

#### Loss of Confidentiality

While we take extensive measures to protect your personal information, there is a minimal risk of loss of confidentiality. We use secure data management systems to protect your data, but no system is completely foolproof.

### Unforeseeable Risks

As with any clinical study, there may be risks that we cannot predict at this time. These unforeseeable risks could arise from the study drugs, procedures, or other aspects of the study. We will monitor your health closely and provide any necessary medical care if unexpected issues arise.

### Conclusion

Your safety is our top priority, and we will take all necessary precautions to minimize these risks. If you have any questions or concerns about the risks involved in this study, please do not hesitate to ask the study team. Your participation is voluntary, and you can withdraw from the study at any time if you feel uncomfortable with the risks.

---

## Benefits

### Potential Benefits for You

Participating in this study may offer you potential benefits, although it is important to understand that the effectiveness of the interventions being tested is not yet known. The primary goal of this study is to explore whether personalized immunomodulation can improve outcomes for children with sepsis-induced multiple organ dysfunction syndrome (MODS). If you are enrolled in the GRACE-2 trial, you may receive GM-CSF, which is being investigated for its potential to reverse immunoparalysis and enhance your immune response. Alternatively, if you are part of the TRIPS trial, you may receive anakinra, which is being studied for its ability to reduce inflammation and improve organ function. These interventions could potentially lead to a reduction in the severity and duration of organ dysfunction, and in some cases, may contribute to a quicker recovery. However, it is crucial to note that these benefits are not guaranteed, and the study is designed to determine the effectiveness of these treatments.

### Potential Benefits for Others

Your participation in this study could contribute significantly to the advancement of medical knowledge and the development of new treatments for sepsis-induced MODS in children. By helping researchers understand how personalized immunomodulation affects immune function and inflammation, you are playing a vital role in potentially improving future care for children facing similar health challenges. The data collected from this study may lead to more effective, targeted therapies that can reduce mortality and morbidity associated with sepsis-induced MODS. Additionally, the insights gained from this research could inform clinical practices and guidelines, ultimately benefiting a broader population of critically ill children. Your involvement is invaluable in paving the way for future innovations in pediatric critical care.

Markdown document has been created: /Users/jmichaeldean/clinical-trials/experimental/informed_consent_document.md


In [25]:
import makeMarkdown
from makeMarkdown import makeMarkdown
makeMarkdown(summaries)

# Full Informed Consent Document

## Study Summary

You are being invited to allow your child to participate in a research study because your child is potentially eligible due to their admission to a Pediatric or Cardiac Intensive Care Unit with sepsis-induced multiple organ dysfunction syndrome (MODS). This study, called the "PeRsonalizEd immunomodulation in pediatriC sepsIS-inducEd MODS (PRECISE)," aims to explore new ways to treat children with this condition by tailoring treatments to their specific immune responses.

Sepsis is a serious condition that occurs when the body's response to an infection causes injury to its own tissues and organs. In some cases, this can lead to MODS, where multiple organ systems in the body stop working properly. Current treatments for sepsis focus on antibiotics and supportive care, but there is a need for more personalized approaches that consider the unique immune responses of each child.

The PRECISE study is designed to test whether personalized treatments can improve outcomes for children with sepsis-induced MODS. The study involves two main trials: the GRACE-2 trial and the TRIPS trial. These trials will test different treatments based on your child's specific immune response. The GRACE-2 trial will focus on children with a suppressed immune response, while the TRIPS trial will focus on those with an overactive immune response.

Participation in this study is entirely voluntary. If you choose to allow your child to participate, they will undergo a series of tests to determine their immune response. Based on these results, they may receive one of the study treatments or be part of an observational group. The study treatments involve medications that are already approved for other uses but are being tested here for their potential to help children with sepsis-induced MODS.

There are potential risks and benefits to participating in this study. The risks include possible side effects from the study medications, such as fever, chills, or changes in blood cell counts. There is also a risk of discomfort from blood draws and other procedures. However, the study team will closely monitor your child to manage any side effects that occur.

The potential benefits of participating include the possibility of improved health outcomes for your child if the study treatments are effective. Additionally, the information gained from this study may help improve treatments for other children with sepsis-induced MODS in the future.

Your decision to allow your child to participate in this study is completely voluntary. You may choose not to participate, or you may withdraw your child from the study at any time without any penalty or loss of benefits to which your child is otherwise entitled. The care your child receives will not be affected by your decision.

If you decide to participate, you will be asked to sign a consent form, and the study team will provide you with more detailed information about the study procedures, potential risks, and benefits. You will have the opportunity to ask any questions you may have before making your decision.

Thank you for considering this important research study. Your participation could contribute to advancing medical knowledge and improving care for children with sepsis-induced MODS.

---

## Background

You are being invited to participate in a research study focused on improving the treatment of a serious condition known as sepsis-induced multiple organ dysfunction syndrome (MODS) in children. Sepsis is a life-threatening condition that occurs when the body's response to an infection causes injury to its own tissues and organs. When sepsis leads to the failure of multiple organs, it is referred to as MODS. This condition is a major cause of illness and death in critically ill children, and current treatments are primarily supportive, such as providing antibiotics and managing symptoms.

The study you are being considered for is called the PRECISE study, which stands for PeRsonalizEd immunomodulation in pediatriC sepsIS-inducEd MODS. This study is part of a larger effort by the Collaborative Pediatric Critical Care Research Network to explore new ways to treat sepsis-induced MODS by tailoring treatments to the specific needs of each child based on their immune system's response to the infection.

The PRECISE study involves two main trials: the GRACE-2 trial and the TRIPS trial. These trials are designed to test different approaches to modulating, or adjusting, the immune system's response in children with sepsis-induced MODS. The goal is to improve outcomes by either boosting the immune system when it is too weak or calming it down when it is too active.

In the GRACE-2 trial, researchers are investigating the use of a drug called GM-CSF (granulocyte-macrophage colony-stimulating factor). GM-CSF is a naturally occurring protein in the body that stimulates the production of white blood cells, which are crucial for fighting infections. The researchers believe that by administering GM-CSF, they can help children whose immune systems are not responding adequately to the infection, a condition known as immunoparalysis. This approach aims to enhance the body's ability to fight off the infection and recover from organ dysfunction.

The TRIPS trial, on the other hand, is focused on children who have a different immune response characterized by excessive inflammation. In this trial, the drug anakinra is being tested. Anakinra is an anti-inflammatory medication that blocks the activity of a specific protein called interleukin-1, which plays a role in the inflammatory response. By using anakinra, the researchers hope to reduce harmful inflammation in children with sepsis-induced MODS, potentially leading to better recovery and outcomes.

The decision to include you in this study is based on specific criteria related to your immune system's response to sepsis. Researchers will conduct tests to determine your immune phenotype, which is a way of categorizing your immune response. Depending on the results, you may be eligible for one of the trials or be part of an observational group that helps researchers understand the condition better without receiving the study drugs.

The scientific rationale behind these interventions is grounded in previous research that has identified distinct immune response patterns in children with sepsis-induced MODS. By targeting these specific patterns with tailored treatments, the study aims to provide more effective care and improve the chances of recovery for children affected by this severe condition.

Your participation in this study could contribute to important scientific knowledge that may lead to better treatments for sepsis-induced MODS in the future.

---

## Number of Participants

This study is being conducted across multiple hospitals within the Collaborative Pediatric Critical Care Research Network (CPCCRN), which includes centers such as Nationwide Children’s Hospital, Children’s Hospital of Philadelphia, and Arkansas Children’s Hospital, among others. The study is funded by the Eunice Kennedy Shriver National Institute of Child Health and Human Development (NICHD). We plan to enroll up to 1,095 participants in total. Specifically, we anticipate enrolling 400 subjects into the GRACE-2 trial and 500 subjects into the TRIPS trial. The study is expected to enroll subjects over a period of four years. Participants will be critically ill children with sepsis-induced multiple organ dysfunction syndrome (MODS), and they will be immunophenotyped to determine eligibility for the interventional trials. This large-scale, multi-center study aims to improve outcomes through personalized, targeted immune modulation.

---

## Study Procedures

### Screening and Enrollment

When you are admitted to the pediatric or cardiac intensive care unit, you will be evaluated to see if you are eligible for our study. This evaluation will involve checking if you meet certain criteria related to your health condition. If you are eligible, we will ask your legal guardian(s) for permission to include you in the study. This initial step is crucial as it helps us determine if you can participate in the study based on your current health status.

### Immune Phenotyping

If you are eligible and your guardian consents, we will perform a process called immune phenotyping. This involves taking a small blood sample from you on the second day after the onset of your condition, known as sepsis-induced multiple organ dysfunction syndrome (MODS). The blood sample will be used to measure certain immune responses and inflammatory markers, such as TNFα production capacity, serum ferritin levels, and C-reactive protein (CRP) levels. These tests help us understand your immune system's response to the illness and determine the best course of action for your treatment within the study.

### Randomization and Study Drug Administration

Based on the results of your immune phenotyping, you may be eligible to receive a study drug. If you qualify, you will be randomly assigned to receive either the study drug or a placebo. This process is called randomization and ensures that the study results are unbiased. The study drug will be administered intravenously, meaning it will be given through a vein, over a period of time. The specific drug and dosage will depend on your immune response and inflammation levels as determined by the phenotyping.

### Follow-up Immune Function Testing

Throughout the study, we will continue to monitor your immune function. This involves taking additional blood samples at specific intervals to measure the same markers as before. These follow-up tests help us track how your immune system is responding to the study drug and whether there are any changes in your condition. The samples will be processed and analyzed to provide insights into your treatment progress.

### Biorepository Sample Collection

As part of the study, we will collect additional biological samples, such as serum, plasma, and possibly endotracheal aspirates if you are intubated. These samples will be stored in a biorepository for future analysis related to the study. The collection of these samples is important for understanding the broader effects of the study drug and the underlying biology of your condition.

### Data Collection and Monitoring

Throughout your participation in the study, we will collect various types of data, including demographic information, eligibility data, and details about your health status and treatment. This data will be securely stored and used to evaluate the effectiveness and safety of the study drug. We will also monitor for any adverse events or changes in your condition that may occur during the study.

### Discontinuation or Withdrawal

You or your guardian may choose to discontinue participation in the study at any time. If you decide to stop participating, we will cease all study-related procedures and data collection. However, we will continue to review any data collected prior to your withdrawal to ensure the integrity of the study.

### Conclusion

These procedures are designed to ensure that we gather accurate and comprehensive information about your condition and the effects of the study drug. Your participation is valuable in helping us understand and improve treatments for sepsis-induced MODS in children.

---

## Alternative Procedures

Your participation in this study is entirely voluntary. You do not need to be part of this study to receive high-quality care for sepsis-induced multiple organ dysfunction syndrome (MODS). The medical team in the Pediatric Intensive Care Unit (PICU) or Cardiac Intensive Care Unit (CICU) will continue to provide state-of-the-art critical care, which includes monitoring and treating sepsis and its complications. This care is based on the best available medical practices and guidelines.

Currently, the monitoring of immune function and the specific treatments being studied, such as personalized immunomodulation, are not part of routine care in the ICU. These investigational treatments aim to improve outcomes by targeting the immune response, but they are not yet standard practice. If you choose not to participate, your child will still receive all the necessary medical interventions to manage sepsis and MODS, including antibiotics, fluid resuscitation, and supportive therapies as needed.

If at any point you decide to withdraw from the study, you can do so without any impact on the care your child receives. The decision to participate or not will not affect your relationship with the medical staff or the quality of care provided. Your child's health and well-being remain the top priority, and the medical team will continue to use all available resources to ensure the best possible outcomes.

---

## Risks

Participating in this study involves several risks that you should be aware of before deciding to take part. These risks are associated with the study drugs, procedures, and data collection. While we have taken steps to minimize these risks, it is important to understand them fully. Additionally, there may be unforeseeable risks that we cannot predict at this time.

### Risks Associated with Study Drugs

#### GM-CSF (Granulocyte Macrophage-Colony Stimulating Factor)

GM-CSF is a medication used to stimulate the production of white blood cells. It is generally safe, but there are some potential risks:

- **Fever and Chills**: You may experience fever and chills as your body responds to the medication.
- **Bone Pain**: Some participants report bone pain, which can be uncomfortable but is usually manageable with pain relief.
- **Dyspnea (Shortness of Breath)**: You might feel short of breath, which should be reported to the study team immediately.
- **Tachycardia (Increased Heart Rate)**: An increased heart rate can occur, which may require monitoring.
- **Peripheral Edema (Swelling)**: Swelling in the limbs can happen, especially if the drug is administered quickly. We will administer it slowly to minimize this risk.
- **Pericardial Effusion (Fluid Around the Heart)**: This is rare but can occur, leading to discomfort or more serious heart issues.
- **Hypersensitivity Reactions**: Allergic reactions, including rash or more severe reactions, can occur. Immediate medical attention will be provided if this happens.

#### Anakinra

Anakinra is used to reduce inflammation by blocking a specific protein in the body. Here are the potential risks:

- **Increased Risk of Infections**: Anakinra can increase the risk of infections, as it affects the immune system. We will monitor you closely for signs of infection.
- **Neutropenia (Low White Blood Cell Count)**: This condition can make you more susceptible to infections. Regular blood tests will be conducted to monitor your white blood cell levels.
- **Hypersensitivity Reactions**: Allergic reactions, including severe reactions like anaphylaxis, are possible. We will be prepared to treat any allergic reactions immediately.

### Risks Associated with Biological Sampling

#### Blood Sampling

Blood samples will be taken several times during the study. The risks associated with blood sampling include:

- **Anemia (Low Red Blood Cell Count)**: Frequent blood draws can lead to anemia, which may cause fatigue and weakness. We will limit the amount of blood drawn to minimize this risk.
- **Pain and Bruising**: You may experience pain and bruising at the site where blood is drawn. This is usually temporary.
- **Infection**: There is a small risk of infection at the site of the blood draw. We will use sterile techniques to minimize this risk.

#### Venipuncture and Capillary Sticks

Additional blood samples may be needed, which could require venipuncture (inserting a needle into a vein) or capillary sticks (pricking the skin). These procedures can cause:

- **Pain and Discomfort**: You may feel pain or discomfort during the procedure.
- **Bruising and Bleeding**: Bruising and minor bleeding can occur at the site of the needle or stick.
- **Infection**: As with blood draws, there is a small risk of infection.

### Risks Associated with Data Collection

#### Loss of Confidentiality

While we take extensive measures to protect your personal information, there is a minimal risk of loss of confidentiality. We use secure data management systems to protect your data, but no system is completely foolproof.

### Unforeseeable Risks

As with any clinical study, there may be risks that we cannot predict at this time. These unforeseeable risks could arise from the study drugs, procedures, or other aspects of the study. We will monitor your health closely and provide any necessary medical care if unexpected issues arise.

### Conclusion

Your safety is our top priority, and we will take all necessary precautions to minimize these risks. If you have any questions or concerns about the risks involved in this study, please do not hesitate to ask the study team. Your participation is voluntary, and you can withdraw from the study at any time if you feel uncomfortable with the risks.

---

## Benefits

### Potential Benefits for You

Participating in this study may offer you potential benefits, although it is important to understand that the effectiveness of the interventions being tested is not yet known. The primary goal of this study is to explore whether personalized immunomodulation can improve outcomes for children with sepsis-induced multiple organ dysfunction syndrome (MODS). If you are enrolled in the GRACE-2 trial, you may receive GM-CSF, which is being investigated for its potential to reverse immunoparalysis and enhance your immune response. Alternatively, if you are part of the TRIPS trial, you may receive anakinra, which is being studied for its ability to reduce inflammation and improve organ function. These interventions could potentially lead to a reduction in the severity and duration of organ dysfunction, and in some cases, may contribute to a quicker recovery. However, it is crucial to note that these benefits are not guaranteed, and the study is designed to determine the effectiveness of these treatments.

### Potential Benefits for Others

Your participation in this study could contribute significantly to the advancement of medical knowledge and the development of new treatments for sepsis-induced MODS in children. By helping researchers understand how personalized immunomodulation affects immune function and inflammation, you are playing a vital role in potentially improving future care for children facing similar health challenges. The data collected from this study may lead to more effective, targeted therapies that can reduce mortality and morbidity associated with sepsis-induced MODS. Additionally, the insights gained from this research could inform clinical practices and guidelines, ultimately benefiting a broader population of critically ill children. Your involvement is invaluable in paving the way for future innovations in pediatric critical care.

Markdown document has been created: /Users/jmichaeldean/clinical-trials/experimental/informed_consent_document.md
